In [1]:
import pandas as pd
from scrape_stats_function import combine_data
pd.set_option('display.max_columns', None)

years = ["2024-2025","2023-2024","2022-2023","2021-2022","2020-2021","2019-2020","2018-2019","2017-2018","2016-2017","2015-2016","2014-2015","2013-2014","2012-2013","2011-2012","2010-2011"]
leagueinfo = [
    {
        "name": "Bundesliga",
        "url": "https://fbref.com/en/comps/20/{year}/stats/{year}-Bundesliga-Stats"
    },
    {
        "name": "Premier League",
        "url": "https://fbref.com/en/comps/9/{year}/stats/{year}-Premier-League-Stats"
    },
    {
        "name": "La Liga",
        "url": "https://fbref.com/en/comps/12/{year}/stats/{year}-La-Liga-Stats"
    },
    {
        "name": "Serie A",
        "url": "https://fbref.com/en/comps/11/{year}/stats/{year}-Serie-A-Stats"
    },
    {
        "name": "Ligue 1",
        "url": "https://fbref.com/en/comps/13/{year}/stats/{year}-Ligue-1-Stats"
    },
    {
        "name": "Eredivisie",
        "url": "https://fbref.com/en/comps/23/{year}/stats/{year}-Eredivisie-Stats"
    },
    {
        "name": "Primeira Liga",
        "url": "https://fbref.com/en/comps/32/{year}/stats/{year}-Primeira-Liga-Stats"
    },
    {
        "name": "Championship",
        "url": "https://fbref.com/en/comps/10/{year}/stats/{year}-EFL-Championship-Stats"
    },
    {
        "name": "Scottish Premiership",
        "url": "https://fbref.com/en/comps/40/{year}/stats/{year}-Scottish-Premiership-Stats"
    }
 
]


In [2]:
combined_list = []
combined_list = combine_data(years, leagueinfo, table_id='stats_standard')

200 https://fbref.com/en/comps/20/2024-2025/stats/2024-2025-Bundesliga-Stats
Found stats_standard in comment for Bundesliga 2024-2025
200 https://fbref.com/en/comps/9/2024-2025/stats/2024-2025-Premier-League-Stats
Found stats_standard in comment for Premier League 2024-2025
200 https://fbref.com/en/comps/12/2024-2025/stats/2024-2025-La-Liga-Stats
Found stats_standard in comment for La Liga 2024-2025
200 https://fbref.com/en/comps/11/2024-2025/stats/2024-2025-Serie-A-Stats
Found stats_standard in comment for Serie A 2024-2025
200 https://fbref.com/en/comps/13/2024-2025/stats/2024-2025-Ligue-1-Stats
Found stats_standard in comment for Ligue 1 2024-2025
200 https://fbref.com/en/comps/23/2024-2025/stats/2024-2025-Eredivisie-Stats
Found stats_standard in comment for Eredivisie 2024-2025
200 https://fbref.com/en/comps/32/2024-2025/stats/2024-2025-Primeira-Liga-Stats
Found stats_standard in comment for Primeira Liga 2024-2025
200 https://fbref.com/en/comps/10/2024-2025/stats/2024-2025-EFL-Cha

In [78]:
df = pd.concat(combined_list,ignore_index=True)


df2 = df[~df['Player'].str.contains("Player", case=False, na=False)]

#df_dropped_players.loc[df_dropped_players['90s'].idxmax()]


In [79]:
pd.set_option('display.max_columns', None)

df2 = df2.drop(['Matches','Rk'], axis=1)

Data Cleaning

In [80]:
int64_col = ['Born','Age','MP','Starts','Gls','Ast','G+A','G-PK','PK','PKatt','CrdY','CrdR','PrgC','PrgP','PrgR','Min']

float64_col = ['90s','xG','xAG','npxG','npxG+xAG','Gls.1','Ast.1','G-PK.1','G+A-PK','xG.1','xAG.1','xG+xAG','npxG.1','npxG+xAG.1','G+A.1']

for col in float64_col:
    df2[col] = pd.to_numeric(df2[col], errors='coerce')

for col in int64_col:
    df2[col] = pd.to_numeric(df2[col], errors='coerce').astype('Int64')

df2 = df2.astype({
    'Player': 'string',
    'Nation': 'string',
    'Pos': 'string',
    'Squad': 'string',
    'League': 'string',
})

In [81]:
new_columns = {
    'MP': 'Matches',
    'G-PK': 'NPK Gls',
    'PK': 'PK Md',
    'PKatt': 'PK Att',
    'CrdY': 'Yel',
    'CrdR': 'Red',
    'PrgC': 'PrgCar',
    'PrgP': 'PrgPas',
    'Gls.1': 'G/90',
    'Ast.1': 'A/90',
    'G+A.1': 'G+A/90',
    'G-PK.1': 'NPK G/90',
    'G+A-PK': 'NPK G+A/90',
    'xG.1': 'xG/90',
    'xAG.1': 'xAG/90',
    'xG+xAG': 'xG+xAG/90',
    'npxG.1': 'npxG/90',
    'npxG+xAG.1': 'npxG+xAG/90',
}

df2 = df2.rename(columns=new_columns)

In [82]:
df2.isnull().sum()

Player            0
Nation           44
Pos               0
Squad             0
Age              37
Born             37
Matches           0
Starts            0
Min               0
90s               0
Gls               0
Ast               0
G+A               0
NPK Gls           0
PK Md             0
PK Att            0
Yel               0
Red               0
xG             1101
npxG           1101
xAG            1101
npxG+xAG       1101
PrgCar         1101
PrgPas         1101
PrgR           1101
G/90              0
A/90              0
G+A/90            0
NPK G/90          0
NPK G+A/90        0
xG/90          1101
xAG/90         1101
xG+xAG/90      1101
npxG/90        1101
npxG+xAG/90    1101
League            0
dtype: int64

In [ ]:
normalize_names = ['Player', 'Squad', 'Pos', 'Nation', 'League','Pos1', 'Pos2']

df2[['Pos1','Pos2']] = df2['Pos'].str.split(',', expand=True)

for col in normalize_names:
    df2[col] = df2[col].str.strip().str.lower()

df2.drop(columns=['Pos1'], inplace=True)

df2.head()

,Player,Nation,Pos,Squad,Age,Born,Matches,Starts,Min,90s,Gls,Ast,G+A,NPK Gls,PK Md,PK Att,Yel,Red,xG,npxG,xAG,npxG+xAG,PrgCar,PrgPas,PrgR,G/90,A/90,G+A/90,NPK G/90,NPK G+A/90,xG/90,xAG/90,xG+xAG/90,npxG/90,npxG+xAG/90,League,Pos2
0,junior adamu,at aut,fw,freiburg,23,2001,25,19,1545,17.2,2,2,4,2,0,0,5,1,5.5,5.5,2.4,7.9,8,14,84,0.12,0.12,0.23,0.12,0.23,0.32,0.14,0.46,0.32,0.46,bundesliga 2024-2025,<NA>
1,karim adeyemi,de ger,"fw,mf",dortmund,22,2002,25,17,1433,15.9,7,6,13,7,0,0,4,0,5.5,5.5,6.0,11.5,68,28,153,0.44,0.38,0.82,0.44,0.82,0.34,0.38,0.72,0.34,0.72,bundesliga 2024-2025,mf
2,amine adli,ma mar,"mf,fw",leverkusen,24,2000,20,6,766,8.5,2,0,2,2,0,0,3,0,2.7,2.7,0.7,3.4,37,24,100,0.23,0.00,0.23,0.23,0.23,0.32,0.08,0.40,0.32,0.40,bundesliga 2024-2025,fw
3,oladapo afolayan,eng eng,"fw,mf",st. pauli,26,1998,32,17,1639,18.2,3,1,4,3,0,0,3,0,3.1,3.1,3.8,7.0,53,32,108,0.16,0.05,0.22,0.16,0.22,0.17,0.21,0.38,0.17,0.38,bundesliga 2024-2025,mf
4,felix agu,ng nga,df,werder bremen,24,1999,22,21,1751,19.5,3,0,3,3,0,0,1,0,1.4,1.4,1.8,3.2,33,43,105,0.15,0.00,0.15,0.15,0.15,0.07,0.09,0.17,0.07,0.17,bundesliga 2024-2025,<NA>


In [60]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16916 entries, 0 to 17577
Data columns (total 38 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       16916 non-null  string 
 1   Nation       16872 non-null  string 
 2   Pos          16916 non-null  string 
 3   Squad        16916 non-null  string 
 4   Age          16879 non-null  Int64  
 5   Born         16879 non-null  Int64  
 6   Matches      16916 non-null  Int64  
 7   Starts       16916 non-null  Int64  
 8   Min          16916 non-null  Int64  
 9   90s          16916 non-null  float64
 10  Gls          16916 non-null  Int64  
 11  Ast          16916 non-null  Int64  
 12  G+A          16916 non-null  Int64  
 13  NPK Gls      16916 non-null  Int64  
 14  PK Md        16916 non-null  Int64  
 15  PK Att       16916 non-null  Int64  
 16  Yel          16916 non-null  Int64  
 17  Red          16916 non-null  Int64  
 18  xG           15815 non-null  float64
 19  npxG     